In [187]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
seed = '10071'
node_num = '300'
shadow = 'shadow10_2'
# sigo recover path
sigo_path = pathlib.Path('../../get_data/recover/' + shadow + '/sigo_recover/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')
#jbr recover path
jbr_path = pathlib.Path('../../get_data/recover/' + shadow + '/jbr/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')

# sigo_path = pathlib.Path('../../ns3.30/data/no_buildings/sigo/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')
# lsgo_path = pathlib.Path('../../ns3.30/data/no_buildings/lsgo/lsgo-seed_' +seed +'nodenum_' +node_num+'.csv')

# csvファイルの取得
pandasを利用：https://note.nkmk.me/python-pandas-value-counts/

In [188]:
sigo_df = pd.read_csv(sigo_path)
jbr_df = pd.read_csv(jbr_path)

In [189]:
sigo_df

,source_x,source_y,recv_x,recv_y,time,recv_priority,hopcount,recv_id,source_id,destination_id,destination_x,destination_y,pri_1,pri_2,pri_3,pri_4,pri_5,pri_recover
0,201,76,198,105,10300200,5,1,60,98,167,565,601,267,180,226,240,60,0
1,201,76,198,153,10300200,2,1,180,98,167,565,601,267,180,226,240,60,0
2,201,76,198,179,10300200,4,1,240,98,167,565,601,267,180,226,240,60,0
3,201,76,201,212,10300200,1,1,267,98,167,565,601,267,180,226,240,60,0
4,201,76,201,218,10300200,3,1,226,98,167,565,601,267,180,226,240,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,401,514,398,608,19310942,1,6,108,255,185,598,652,108,104,198,16,10000000,0
507,398,608,406,600,19311490,2,7,16,108,185,598,652,167,16,10000000,10000000,10000000,0
508,398,608,448,601,19311490,1,7,167,108,185,598,652,167,16,10000000,10000000,10000000,0
509,448,601,600,601,19312039,1,8,19,167,185,598,652,19,96,156,90,137,0


In [190]:
jbr_df

,source_x,source_y,recv_x,recv_y,time,recv_priority,hopcount,recv_id,source_id,destination_id,destination_x,destination_y,pri_1,pri_2,pri_3,pri_4,pri_5,pri_recover
0,201,76,198,105,10300200,5,1,60,98,167,565,601,267,180,226,240,60,0
1,201,76,198,153,10300200,2,1,180,98,167,565,601,267,180,226,240,60,0
2,201,76,198,179,10300200,4,1,240,98,167,565,601,267,180,226,240,60,0
3,201,76,201,212,10300200,1,1,267,98,167,565,601,267,180,226,240,60,0
4,201,76,201,218,10300200,3,1,226,98,167,565,601,267,180,226,240,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,401,514,401,573,19302394,2,5,104,255,185,598,652,108,104,198,16,10000000,0
362,401,514,406,600,19302394,4,5,16,255,185,598,652,108,104,198,16,10000000,0
363,401,514,398,608,19302394,1,5,108,255,185,598,652,108,104,198,16,10000000,0
364,398,608,406,600,19302942,2,6,16,108,185,598,652,167,16,10000000,10000000,10000000,0


# recovery packet の多さ

In [191]:
jbr_df['pri_recover'].value_counts().to_dict()

{0: 362, 1: 4}

In [192]:
sigo_df['pri_recover'].value_counts().to_dict()

{0: 446, 1: 65}

# destination_idごとのパケット数


destination_id を重複なしにリスト保存

In [193]:
sigo_df['destination_id'].value_counts().to_dict()

{192: 89,
 142: 75,
 69: 66,
 57: 61,
 135: 52,
 90: 50,
 167: 39,
 87: 31,
 185: 28,
 33: 20}

In [194]:
jbr_df['destination_id'].value_counts().to_dict()

{142: 86,
 192: 80,
 69: 52,
 167: 39,
 87: 29,
 185: 21,
 90: 20,
 57: 18,
 33: 14,
 135: 7}

In [195]:
dest_list = sigo_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

In [196]:
dest_list = jbr_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

sigoとlsgoそれぞれ目的地に届いたidだけをlistに保存

In [197]:
sigo_reach_list = sigo_df[sigo_df['recv_id'] == sigo_df['destination_id']]['destination_id'].value_counts().index.tolist()
jbr_reach_list = jbr_df[jbr_df['recv_id'] == jbr_df['destination_id']]['destination_id'].value_counts().index.tolist()
sigo_reach_list.sort()
jbr_reach_list.sort()

print('sigo:',sigo_reach_list)
print('jbr:',jbr_reach_list)

sigo: [33, 69, 87, 135, 142, 185, 192]
jbr: [69, 87]


# 平均delayを返すメソッド

In [198]:
def packet_delay(path, spefic=0): #spefic 0 or id   0の時平均を返す idのときそのdestination id のdelayを返す
    delay_list = []
    #csvファイルの取得
    path_df = pd.read_csv(path)
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    if(spefic == 0):
        #reach_listを回す
        for reach_id in reach_list:
            source_time = path_df[path_df['destination_id'] == reach_id]['time']
            recv_time = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['time']
            delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
            delay = delay / 100000
        #         print('desID:', reach_id, 'delay:',delay)
            delay_list.append(delay)
        delay_mean = np.mean(delay_list)
        return delay_mean
    else:
        source_time = path_df[path_df['destination_id'] == spefic]['time']
        recv_time = path_df[(path_df['destination_id'] == spefic) & (path_df['destination_id'] == path_df['recv_id'])]['time']
        delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
        delay = delay / 100000
        return delay

In [199]:
packet_delay(sigo_path)

0.25409714285714285

In [200]:
packet_delay(jbr_path)

0.32306

# 軌跡描画

In [201]:
def draw_source_area(log, img_area):
    cv2.rectangle(img_area, (log.source_x - 15, log.source_y - 15), (log.source_x + 15, log.source_y + 15), (255, 255, 0),thickness=8)
    cv2.rectangle(img_area, (log.destination_x - 15, log.destination_y - 15), (log.destination_x + 15, log.destination_y + 15), (255, 255, 0),thickness=8)
    #source nodeを視覚で囲む
    cv2.rectangle(img_area, (log.source_x - 15, log.source_y - 15), (log.source_x + 15, log.source_y + 15), (255, 255, 0),thickness=8)

    #方眼
    plt.figure(figsize=(10, 10))
    print('グラフサイズ')
    plt.grid(color='w', linestyle='dotted', linewidth=1.0)
    plt.xlim([0, 1000])
    plt.xticks([1000,800,600,400,200, 0]) 
    plt.ylim([0, 1000])
    plt.yticks([1000,800,600,400,200, 0])  #メモリの間隔を指定 
    print('source_area 描画', log.source_x)

# パケットの軌跡を描画
def draw(log,count,img_area,protocol,text):
    recover_line_color = (255, 0, 0)
    if protocol == 'sigo':
        line_color = (0, 255, 0)
        thick = 7
        des_thick = 8
    elif protocol == 'jbr':
        line_color = (0, 255, 255)
        thick = 6
        des_thick = 2
    else:
        line_color = (255, 0, 0)
        thick = 2
        des_thick = 2
        

    if(log.pri_recover == 1):
        cv2.arrowedLine(img_area,(log.source_x, log.source_y), 
                            (log.recv_x, log.recv_y), recover_line_color, thickness=thick, tipLength=0.3)
        cv2.circle(img=img_area, center=(log.source_x,
                               log.source_y), radius=5, color=(255,0,0), thickness=-1)
        cv2.circle(img=img_area, center=(log.recv_x,
                               log.recv_y), radius=5, color=(255,0,0), thickness=-1)
    else:
        cv2.arrowedLine(img_area,(log.source_x, log.source_y), 
                            (log.recv_x, log.recv_y), line_color, thickness=thick, tipLength=0.1)
        cv2.circle(img=img_area, center=(log.source_x,
                               log.source_y), radius=5, color=(255,0,0), thickness=-1)
        cv2.circle(img=img_area, center=(log.recv_x,
                               log.recv_y), radius=5, color=(255,0,0), thickness=-1)
    
        
    
    
    #受け取ったのが宛先だったならば
    if log.recv_id == log.destination_id:
        cv2.drawMarker(img_area, (log.recv_x, log.recv_y), line_color, thickness=des_thick, markerType=cv2.MARKER_TILTED_CROSS, markerSize=25)
    if text == 'あり':    
        cv2.putText(img_area, str(log.source_id), (log.source_x + 10,log.source_y +10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), thickness=2)
        cv2.putText(img_area, str(log.recv_id), (log.recv_x + 10,log.recv_y +10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), thickness=2)

In [202]:
img_area = np.zeros(shape=(1000,1000, 3), dtype=np.int16)
protocol_list = ["sigo", "jbr", "both"]
text_list = ["なし", "あり"]
print('sigo:',sigo_reach_list,'PDR:',len(sigo_reach_list)/len(sigo_df['destination_id'].value_counts().index.tolist()))
print('jbr:',jbr_reach_list,'PDR:',len(jbr_reach_list)/len(jbr_df['destination_id'].value_counts().index.tolist()))
print('sigo delay:', packet_delay(sigo_path))
print('jbr delay:', packet_delay(jbr_path))

#packetの軌跡を描画 
@interact(minHop=(0,19,1), maxHop=(1,21, 1), desId=dest_list, protocol = protocol_list, text = text_list)
def packetDraw(protocol, desId, minHop=0, maxHop = 21, text = "なし"):
    
    img_area = np.zeros(shape=(2100,2100, 3), dtype=np.int16)
    count = 1
    if protocol == 'sigo':     
        for log in sigo_df[sigo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
                
        plt.imshow(img_area)
    
    elif protocol == 'jbr':
        
        for log in jbr_df[jbr_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
 
        plt.imshow(img_area)
    
    else:
        if(desId in sigo_reach_list):
            print('sigo packet delay id', desId, 'delay', packet_delay(sigo_path, desId))
            
        if(desId in jbr_reach_list):
            print('jbr packet delay id', desId, 'delay', packet_delay(jbr_path, desId))
        
        for log in sigo_df[sigo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,'sigo',text)
                
        for log in jbr_df[jbr_df['destination_id'] == desId].itertuples():
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,'lsgo',text)
                
        plt.imshow(img_area)

sigo: [33, 69, 87, 135, 142, 185, 192] PDR: 0.7
jbr: [69, 87] PDR: 0.2
sigo delay: 0.25409714285714285
jbr delay: 0.32306


interactive(children=(Dropdown(description='protocol', options=('sigo', 'jbr', 'both'), value='sigo'), Dropdow…

210 344

In [39]:
def output_df(hopcount, des_id):
    print('lsgo')
    display(lsgo_df[(lsgo_df['hopcount'] == hopcount) & (lsgo_df['destination_id'] == des_id)])
    print('sigo')
    display(sigo_df[(sigo_df['hopcount'] == hopcount) & (sigo_df['destination_id'] == des_id)])

In [314]:
@interact(hopcount=(0,21,1), des_id = dest_list)
def df_run(hopcount, des_id):
    output_df(hopcount, des_id)

interactive(children=(IntSlider(value=10, description='hopcount', max=21), Dropdown(description='des_id', opti…